# Project: Healthcare Data Integration and Analysis

This template provides a starting point to help you complete the Tasks 3 and 4 of the project. The questions in this template are numbered in alignment with the Project Overview reading and submission template.
You can use it as-is, modify it, or create your own structure; just be sure to complete all the required tasks.

In [2]:
# Install required packages
import sqlite3
import requests
import pandas as pd
from pathlib import Path

# Task 3: Identify and fix the issues in a data set using Python

In [23]:
#load dataset
df = pd.read_csv("https://foundations-of-healthcare-data-analytics-4e579d.gitlab.io/labs/Projects/Patients_EHR_raw_data.csv")

In [24]:
#print top 5 rows
df.head()

,PatientID,DateOfBirth,Gender,ZipCode,PrimaryCondition,AdmissionDate
0,NV001,1983-03-11,Female,30933.0,Type 2 Diabetes (E11),07-06-2024
1,NV004,NaN,Male,76999.0,Asthma (J45.909),27-10-2022
2,NV005,1984-08-25,Female,40307.0,Osteoarthritis (M15.9),30-10-2024
3,NV006,1982-12-16,Female,56844.0,Type 2 Diabetes (E11),05-07-2022
4,NV007,1990-10-11,Male,NaN,Hyperlipidemia (E78.5),17-06-2024


### 3.1a. Write the Python code to identify the Number of missing values.

In [25]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
PatientID           0
DateOfBirth         3
Gender              0
ZipCode             4
PrimaryCondition    0
AdmissionDate       0
dtype: int64


### 3.1b. Write the Python code to identify the number of duplicate rows.

In [26]:
# Check for duplicate rows
dup_rows = df.duplicated(keep=False)
print(f"\nNumber of duplicate rows: {dup_rows.sum()}")
if dup_rows.any():
    display(df[dup_rows])


Number of duplicate rows: 4


,PatientID,DateOfBirth,Gender,ZipCode,PrimaryCondition,AdmissionDate
20,NV028,18-05-1978,Male,NaN,Type 2 Diabetes (E11),27-10-2022
21,NV028,18-05-1978,Male,NaN,Type 2 Diabetes (E11),27-10-2022
43,NV053,1941-04-03,Male,39856.0,Hyperlipidemia (E78.5),17-06-2024
44,NV053,1941-04-03,Male,39856.0,Hyperlipidemia (E78.5),17-06-2024


### 3.1c Write the Python code to identify inconsistent categorical entries in the Gender column.

In [27]:
# Identify inconsistent categorical entries
print("\nUnique values in Gender column:")
print(df['Gender'].unique())


Unique values in Gender column:
['Female' 'Male' 'M' 'male' 'F']


### 3.1d Write the Python code to identify inconsistent date formats in the AdmissionDate column.

In [90]:
# Check date format inconsistencies
print("\nSample of AdmissionDate values (showing mixed formats):")
df['cleaned_dates']=pd.to_datetime(df['AdmissionDate'],errors='coerce')
inconsistent_rows=df[df['cleaned_dates'].isnull()]
print("\nInconsistent Rows:")
print(inconsistent_rows)


Sample of AdmissionDate values (showing mixed formats):

Inconsistent Rows:
   PatientID DateOfBirth Gender  ZipCode        PrimaryCondition  \
5      NV008  1949-07-08   Male  56844.0  Hyperlipidemia (E78.5)   
37     NV046  1949-07-07   Male  26372.0        Asthma (J45.909)   

   AdmissionDate cleaned_dates  
5          45258           NaT  
37      45439.00           NaT  


### 3.2a Write the Python code to handle missing values appropriately.

In [29]:
# Create a working copy - keep original data untouched for reference
df_clean = df.copy()
# Fill missing categorical values with 'Unknown'
df_clean['DateOfBirth'] = df_clean['DateOfBirth'].fillna('Unknown')
# Fill missing categorical values with 'Unknown'
df_clean['ZipCode'] = df_clean['ZipCode'].fillna('Unknown')
# Check for missing values
print("\nMissing values per column:")
print(df_clean.isnull().sum())


Missing values per column:
PatientID           0
DateOfBirth         0
Gender              0
ZipCode             0
PrimaryCondition    0
AdmissionDate       0
dtype: int64


### 3.2b Write the Python code to remove exact duplicate rows.

In [30]:
# Create a working copy - keep original data untouched for reference
df_clean = df.copy()
print("Working copy created. Original data preserved.")
# Count duplicates before removal
duplicates_before = df_clean.duplicated().sum()
rows_before = len(df_clean)

# Remove exact duplicate rows (keep first occurrence)
df_clean = df_clean.drop_duplicates(keep='first')

# Report results
rows_after = len(df_clean)
print(f"Rows before deduplication: {rows_before}")
print(f"Duplicate rows found: {duplicates_before}")
print(f"Rows after deduplication: {rows_after}")
print(f"Rows removed: {rows_before - rows_after}")

Working copy created. Original data preserved.
Rows before deduplication: 74
Duplicate rows found: 2
Rows after deduplication: 72
Rows removed: 2


### 3.2c Write the Python code to standardize gender values.

In [31]:
# Standardize Gender column
print("Before standardization - Gender unique values:")
print(df_clean['Gender'].value_counts(dropna=False))

# Convert to lowercase and strip whitespace for consistent matching
df_clean['Gender'] = df_clean['Gender'].astype(str).str.strip().str.lower()

# Define mapping for known variations
gender_map = {
    'male': 'Male', 'm': 'Male',
    'female': 'Female', 'f': 'Female',
}

# Apply mapping
df_clean['Gender'] = df_clean['Gender'].map(
    lambda x: gender_map.get(x, x.capitalize() if pd.notna(x) else x)
)

print("\nAfter standardization - Gender unique values:")
print(df_clean['Gender'].value_counts(dropna=False))

Before standardization - Gender unique values:
Female    34
Male      32
F          4
M          1
male       1
Name: Gender, dtype: int64

After standardization - Gender unique values:
Female    38
Male      34
Name: Gender, dtype: int64


### 3.2d Write the Python code to standardize date values.

In [34]:
# Parse dates with mixed formats (YYYY-MM-DD and DD/MM/YYYY)
# AdmissionDate

df_clean['AdmissionDate_parsed'] = pd.to_datetime(
    df_clean['AdmissionDate'],
    errors='coerce',  # Convert unparseable dates to NaT (Not a Time)
    dayfirst=True     # Assume day comes first in ambiguous formats
)

print("Date parsing results:")
print(f"Successfully parsed: {df_clean['AdmissionDate_parsed'].notna().sum()} dates")
print(f"Failed to parse: {df_clean['AdmissionDate_parsed'].isna().sum()} dates")

print("\nSample of original vs parsed dates:")
display(df_clean[['AdmissionDate', 'AdmissionDate_parsed']].head(10))

# Parse dates with mixed formats (YYYY-MM-DD and DD/MM/YYYY)
# DateOfBirth


df_clean['DateOfBirth_parsed'] = pd.to_datetime(
    df_clean['DateOfBirth'],
    errors='coerce',  # Convert unparseable dates to NaT (Not a Time)
    dayfirst=True     # Assume day comes first in ambiguous formats
)

print("Date parsing results:")
print(f"Successfully parsed: {df_clean['DateOfBirth_parsed'].notna().sum()} dates")
print(f"Failed to parse: {df_clean['DateOfBirth_parsed'].isna().sum()} dates")

print("\nSample of original vs parsed dates:")
display(df_clean[['DateOfBirth', 'DateOfBirth_parsed']].head(10))




Date parsing results:
Successfully parsed: 70 dates
Failed to parse: 2 dates

Sample of original vs parsed dates:


,AdmissionDate,AdmissionDate_parsed
0,07-06-2024,2024-06-07
1,27-10-2022,2022-10-27
2,30-10-2024,2024-10-30
3,05-07-2022,2022-07-05
4,17-06-2024,2024-06-17
5,45258,NaT
6,09-05-2024,2024-05-09
7,25-06-2023,2023-06-25
8,21-09-2024,2024-09-21
9,07-07-2022,2022-07-07


Date parsing results:
Successfully parsed: 69 dates
Failed to parse: 3 dates

Sample of original vs parsed dates:


,DateOfBirth,DateOfBirth_parsed
0,1983-03-11,1983-03-11
1,NaN,NaT
2,1984-08-25,1984-08-25
3,1982-12-16,1982-12-16
4,1990-10-11,1990-10-11
5,1949-07-08,1949-07-08
6,1955-07-11,1955-07-11
7,1944-11-11,1944-11-11
8,1975-01-14,1975-01-14
9,1943-08-25,1943-08-25


# Task 4: Analyze the data sets using SQL

## Medical Data SQLite Exploration
Before you write the queries listed in Task 4 of the project, execute the cells in this section to inspect table structure and query sample rows.

In [43]:
db_url = "https://foundations-of-healthcare-data-analytics-4e579d.gitlab.io/labs/medical_data.db"  # replace
db_path = Path("medical_data.db")

In [44]:
response = requests.get(db_url)
response.raise_for_status()
db_path.write_bytes(response.content)
print("Downloaded to", db_path)

Downloaded to medical_data.db


In [46]:
#conn = sqlite3.connect(db_path)
conn=sqlite3.connect("medical_data.db")
cur = conn.cursor()

In [47]:
def run_query(query: str):
    df = pd.read_sql_query(query, conn)
    display(df)

### Table structure: pharmacy_claims

In [48]:
run_query("PRAGMA table_info(pharmacy_claims);")

,cid,name,type,notnull,dflt_value,pk
0,0,ClaimID,TEXT,0,None,0
1,1,PatientID,TEXT,0,None,0
2,2,Medication,TEXT,0,None,0
3,3,Dosage,TEXT,0,None,0
4,4,FillDate,TIMESTAMP,0,None,0
5,5,Payer,TEXT,0,None,0
6,6,ClinicID,TEXT,0,None,0
7,7,ChargeAmount,REAL,0,None,0
8,8,PaidAmount,REAL,0,None,0


### Table structure: lab_results

In [49]:
run_query("PRAGMA table_info(lab_results);")

,cid,name,type,notnull,dflt_value,pk
0,0,PatientID,TEXT,0,None,0
1,1,LabTestID,TEXT,0,None,0
2,2,CollectionDate,TIMESTAMP,0,None,0
3,3,TestName,TEXT,0,None,0
4,4,TestResultValue,REAL,0,None,0
5,5,Units,TEXT,0,None,0
6,6,ReferenceRangeLow,REAL,0,None,0
7,7,ReferenceRangeHigh,REAL,0,None,0
8,8,AbnormalFlag,TEXT,0,None,0


### Table structure: patients

In [50]:
run_query("PRAGMA table_info(patients);")

,cid,name,type,notnull,dflt_value,pk
0,0,PatientID,TEXT,0,None,0
1,1,DateOfBirth,TIMESTAMP,0,None,0
2,2,Gender,TEXT,0,None,0
3,3,ZipCode,INTEGER,0,None,0
4,4,PrimaryCondition,TEXT,0,None,0
5,5,AdmissionDate,TIMESTAMP,0,None,0


### Top 5 patients

In [51]:
run_query("SELECT * FROM patients LIMIT 5;")

,PatientID,DateOfBirth,Gender,ZipCode,PrimaryCondition,AdmissionDate
0,NV094,1962-11-05 00:00:00,Female,95982,Osteoarthritis (M15.9),2023-11-06 00:00:00
1,NV027,1997-12-28 00:00:00,Male,70714,Asthma (J45.909),2024-10-30 00:00:00
2,NV005,1984-08-25 00:00:00,Female,40307,Osteoarthritis (M15.9),2024-10-30 00:00:00
3,NV031,1976-07-08 00:00:00,Female,26647,Asthma (J45.909),2024-08-07 00:00:00
4,NV008,1949-07-08 00:00:00,Male,56844,Hyperlipidemia (E78.5),2023-11-28 00:00:00


## Write the SQL queries and execute them.
The questions are numbered in alignment with the Project Overview reading and submission template.

### 4.1a Write an SQL query to list all patients born before 1980.  

In [68]:
run_query("SELECT * FROM patients WHERE strftime('%Y',DateOfBirth) < '1980';")


,PatientID,DateOfBirth,Gender,ZipCode,PrimaryCondition,AdmissionDate
0,NV094,1962-11-05 00:00:00,Female,95982,Osteoarthritis (M15.9),2023-11-06 00:00:00
1,NV031,1976-07-08 00:00:00,Female,26647,Asthma (J45.909),2024-08-07 00:00:00
2,NV008,1949-07-08 00:00:00,Male,56844,Hyperlipidemia (E78.5),2023-11-28 00:00:00
3,NV046,1949-07-07 00:00:00,Male,26372,Asthma (J45.909),2024-05-27 00:00:00
4,NV089,1943-07-18 00:00:00,Female,87372,Osteoarthritis (M15.9),2024-01-12 00:00:00
5,NV029,1976-08-31 00:00:00,Female,41617,Asthma (J45.909),2022-05-21 00:00:00
6,NV017,1961-07-27 00:00:00,Male,14441,Hyperlipidemia (E78.5),2022-02-12 00:00:00
7,NV012,1966-07-24 00:00:00,Male,67468,Hypertension (I10),2023-12-07 00:00:00
8,NV010,1947-11-12 00:00:00,Female,10156,Osteoarthritis (M15.9),2023-05-13 00:00:00
9,NV076,1956-02-09 00:00:00,Female,14863,Asthma (J45.909),2024-02-27 00:00:00


### 4.1b Write an SQL query to list all female patients with Type 2 Diabetes.

In [73]:
run_query("SELECT * FROM patients WHERE PrimaryCondition='Type 2 Diabetes (E11)' AND Gender='Female'")

,PatientID,DateOfBirth,Gender,ZipCode,PrimaryCondition,AdmissionDate
0,NV072,1992-11-01 00:00:00,Female,12050,Type 2 Diabetes (E11),2023-09-20 00:00:00
1,NV001,1983-03-11 00:00:00,Female,30933,Type 2 Diabetes (E11),2024-06-07 00:00:00
2,NV061,1986-07-06 00:00:00,Female,48060,Type 2 Diabetes (E11),2024-04-20 00:00:00
3,NV081,1968-12-19 00:00:00,Female,19045,Type 2 Diabetes (E11),2024-03-25 00:00:00
4,NV086,1971-08-06 00:00:00,Female,54160,Type 2 Diabetes (E11),2022-04-18 00:00:00
5,NV062,1957-01-02 00:00:00,Female,37775,Type 2 Diabetes (E11),2024-11-14 00:00:00


### 4.1c Write an SQL query to display all lab tests performed. List each patient (PatientID, Gender, and DateOfBirth from *patients*) with their Lab test name and result value (TestName and TestResultValue from *lab_results*).

In [80]:

#load dataset
df_lab_results = pd.read_csv("lab_results.csv")
df_lab_results.to_sql('lab_results',conn,if_exists='replace',index=False)
run_query("SELECT patients.PatientID, patients.Gender, patients.DateOfBirth, lab_results.TestName,lab_results.TestResultValue FROM patients LEFT Join lab_results on patients.PatientID=lab_results.PatientID")

,PatientID,Gender,DateOfBirth,TestName,TestResultValue
0,NV094,Female,1962-11-05 00:00:00,Creatinine,171.16
1,NV094,Female,1962-11-05 00:00:00,HbA1c,169.57
2,NV094,Female,1962-11-05 00:00:00,HbA1c,187.10
3,NV027,Male,1997-12-28 00:00:00,HbA1c,164.21
4,NV027,Male,1997-12-28 00:00:00,HbA1c,169.52
...,...,...,...,...,...
196,NV065,Male,1984-09-09 00:00:00,HbA1c,1.06
197,NV065,Male,1984-09-09 00:00:00,LDL Cholesterol,1.04
198,NV075,Female,1944-08-03 00:00:00,Creatinine,1.01
199,NV075,Female,1944-08-03 00:00:00,HbA1c,1.00


### 4.1d Write an SQL query to display all distinct patients with their payer(s). List each patient (PatientID, Gender, and DateofBirth from *patients*) with their payer(s) from *pharmacy_claims*).

In [83]:
#load dataset
df_pharmacy_claims = pd.read_csv("Pharmacy_Claims.csv")
df_pharmacy_claims.to_sql('lab_results',conn,if_exists='replace',index=False)
run_query("SELECT patients.PatientID, patients.Gender, patients.DateOfBirth, pharmacy_claims.Payer from patients LEFT JOIN pharmacy_claims on patients.PatientID=pharmacy_claims.PatientID;")

,PatientID,Gender,DateOfBirth,Payer
0,NV094,Female,1962-11-05 00:00:00,Medicare
1,NV094,Female,1962-11-05 00:00:00,Medicare
2,NV027,Male,1997-12-28 00:00:00,Medicaid
3,NV027,Male,1997-12-28 00:00:00,Medicaid
4,NV027,Male,1997-12-28 00:00:00,Medicaid
...,...,...,...,...
95,NV059,Male,1959-11-20 00:00:00,Medicare
96,NV004,Female,1959-05-27 00:00:00,Private
97,NV065,Male,1984-09-09 00:00:00,Medicaid
98,NV075,Female,1944-08-03 00:00:00,Medicaid
